# 요인분석 : 특이케이스

In [2]:
import numpy as np
import pandas as pd
from scipy.stats import wishart, multivariate_normal
from sklearn.preprocessing import StandardScaler
from factor_analyzer import FactorAnalyzer, Rotator
import matplotlib.pyplot as plt
from scipy.linalg import block_diag

# 시계열 구조 (1)
* 공분산행렬이 AR(1) 구조인 경우
* 우선 자료부터 생성해보자

In [3]:
# 자료 만들기 : 시계열구조
pp = 3

def AR_cov(r, p):
    return r**np.abs(np.subtract.outer(np.arange(p), np.arange(p)))

np.random.seed(2022)
m1 = wishart.rvs(df=pp, scale=AR_cov(0.9, pp), size=1)
m2 = wishart.rvs(df=pp, scale=AR_cov(0.1, pp), size=1)

In [4]:
print("\nMatrix m1:")
print(pd.DataFrame(np.round(m1, 2)))


Matrix m1:
      0     1     2
0  9.78  8.80  7.55
1  8.80  8.32  7.11
2  7.55  7.11  6.71


In [5]:
print("\nMatrix m2:")
print(pd.DataFrame(np.round(m2, 2)))


Matrix m2:
      0     1     2
0  0.06  0.09  0.10
1  0.09  2.07  0.23
2  0.10  0.23  1.56


# 시계열 구조 (2)
* 2개의 집단 : 일반 + 시계열 -> 겹친 형태의 행렬

In [6]:
combined_matrix = block_diag(np.identity(3), AR_cov(0.1, 3))
print("\nCombined Matrix:")
print(pd.DataFrame(np.round(combined_matrix, 2)))


Combined Matrix:
     0    1    2     3    4     5
0  1.0  0.0  0.0  0.00  0.0  0.00
1  0.0  1.0  0.0  0.00  0.0  0.00
2  0.0  0.0  1.0  0.00  0.0  0.00
3  0.0  0.0  0.0  1.00  0.1  0.01
4  0.0  0.0  0.0  0.10  1.0  0.10
5  0.0  0.0  0.0  0.01  0.1  1.00


In [7]:
pp = 8
Sigma = wishart.rvs(df=pp, scale=np.identity(pp), size=1)
mu = np.zeros(pp)

# 인접 집단
pp = 8
n = 1000
mu = np.zeros(pp)
d1 = multivariate_normal.rvs(mean=mu, cov=Sigma, size=n)
d1 = pd.DataFrame(StandardScaler().fit_transform(d1), columns=[f'X{i}' for i in range(1, pp + 1)])

fa_adj = FactorAnalyzer(n_factors=3, rotation='varimax')
fa_adj.fit(d1.drop(columns='X4'))

print("\nFactor Analysis for adjacent groups (Varimax rotation):")
print(fa_adj.loadings_)


Factor Analysis for adjacent groups (Varimax rotation):
[[ 0.74606624 -0.04840634  0.15029679]
 [ 0.30026933 -0.2710685   0.91586555]
 [-0.38652636 -0.64137004 -0.06661851]
 [-0.04394951  0.74877946 -0.22467767]
 [-0.47340174 -0.05274745  0.66805166]
 [ 0.04337934  0.70020816 -0.12958172]
 [ 0.90311973  0.38246159 -0.23569236]]
